In [1]:
# tree structure finding from database
import json
from pymongo import MongoClient

In [28]:
client = MongoClient('localhost', 27017)
db = client['thesis']
dataset = "quora"
algo_list = ['lr_mvdt', 'rs_mvdt']

In [32]:
tree_info = [{'algorith': 'lr_mvdt','zero_depth':0, 'perfect_balance':0, 'almost_balance': 0, 'left_skewed': 0, 'right_skewed': 0}, 
             {'algorith': 'rs_mvdt','zero_depth':0, 'perfect_balance':0, 'almost_balance': 0, 'left_skewed': 0, 'right_skewed': 0}]

for idx, algo in enumerate(algo_list):
    query = [
        {'$match': {'dataset': dataset, 'algorithm': algo}},
        {'$project': {'dataset':1, 'algorithm': 1, 'branch_sizes':1, 'max_depth':1, 'accuracy':1,'precision':1, 'recall':1}}
    ]

    query_result = db.quora.aggregate(query)
    total = 0
    for result in query_result:
        if result['accuracy'][0] <= 0 and result['accuracy'][1] <= 0:
            print(result['branch_sizes'][0], result['branch_sizes'][1], result['accuracy'], result['precision'], result['recall'])
        total += 1
        if result['max_depth'] == 0:
            tree_info[idx]['zero_depth'] += 1
        else:
            total = result['branch_sizes'][0] + result['branch_sizes'][1]
            left= result['branch_sizes'][0]
            right = result['branch_sizes'][1]
            
            left_percent = (left/total)*100
            right_percent = (right/total)*100
            
            difference = left_percent - right_percent
            if difference == 0:
                # perfect balance condition
                tree_info[idx]['perfect_balance'] += 1
            elif abs(difference) < 10:
                # balance tree
                tree_info[idx]['almost_balance'] += 1
            elif difference > 0:
                # if positive number than lef skewed since right is subtracted from left
                tree_info[idx]['left_skewed'] += 1
            elif difference < 0:
                tree_info[idx]['right_skewed'] += 1
                # if negative number than right skewed
            else:
                print("Some thing worng", result['branch_sizes'])
        
#     print(total)

In [25]:
tree_info

[{'algorith': 'lr_mvdt',
  'zero_depth': 0,
  'perfect_balance': 0,
  'almost_balance': 0,
  'left_skewed': 0,
  'right_skewed': 0},
 {'algorith': 'rs_mvdt',
  'zero_depth': 0,
  'perfect_balance': 0,
  'almost_balance': 0,
  'left_skewed': 0,
  'right_skewed': 0}]